<h1>4장 텍스트 분류</h1>
<i>표현 모델과 생성 모델을 사용해 텍스트 분류하기</i>

<a href="https://github.com/rickiepark/handson-llm"><img src="https://img.shields.io/badge/GitHub%20Repository-black?logo=github"></a>
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rickiepark/handson-llm/blob/main/chapter04.ipynb)

---

이 노트북은 <[핸즈온 LLM](https://tensorflow.blog/handson-llm/)> 책 4장의 코드를 담고 있습니다.

---

<a href="https://tensorflow.blog/handson-llm/">
<img src="https://tensorflow.blog/wp-content/uploads/2025/05/ed95b8eca688ec98a8_llm.jpg" width="350"/></a>

### [선택사항] - <img src="https://colab.google/static/images/icons/colab.png" width=100>에서 패키지 선택하기


이 노트북을 구글 코랩에서 실행한다면 다음 코드 셀을 실행하여 이 노트북에서 필요한 패키지를  설치하세요.

---

💡 **NOTE**: 이 노트북의 코드를 실행하려면 GPU를 사용하는 것이 좋습니다. 구글 코랩에서는 **런타임 > 런타임 유형 변경 > 하드웨어 가속기 > T4 GPU**를 선택하세요.

---


In [ ]:
%%capture
!pip install datasets

In [ ]:
# 깃허브에서 위젯 상태 오류를 피하기 위해 진행 표시줄을 나타내지 않도록 설정합니다.
from transformers.utils import logging

logging.disable_progress_bar()

# 영화 리뷰 데이터셋

In [ ]:
from datasets import load_dataset

# 데이터를 로드합니다.
data = load_dataset("rotten_tomatoes")
data

In [ ]:
data["train"][0, -1]

# 표현 모델로 텍스트 분류하기

# 작업에 특화된 모델 사용하기

In [ ]:
from transformers import pipeline

# 허깅 페이스 모델 경로
model_path = "cardiffnlp/twitter-roberta-base-sentiment-latest"

# 파이프라인으로 모델을 로드합니다.
pipe = pipeline(
    model=model_path,
    tokenizer=model_path,
    return_all_scores=True,
    device="cuda:0"
)

In [ ]:
import numpy as np
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset

# 추론을 실행합니다.
y_pred = []
for output in tqdm(pipe(KeyDataset(data["test"], "text")), total=len(data["test"])):
    negative_score = output[0]["score"]
    positive_score = output[2]["score"]
    assignment = np.argmax([negative_score, positive_score])
    y_pred.append(assignment)

In [ ]:
from sklearn.metrics import classification_report

def evaluate_performance(y_true, y_pred):
    """분류 리포트를 만들어 출력합니다."""
    performance = classification_report(
        y_true, y_pred,
        target_names=["Negative Review", "Positive Review"]
    )
    print(performance)

In [ ]:
evaluate_performance(data["test"]["label"], y_pred)

# 임베딩을 활용하여 분류 작업 수행하기

## 지도 학습 분류

In [ ]:
from sentence_transformers import SentenceTransformer

# 모델을 로드합니다.
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# 텍스트를 임베딩으로 변환합니다.
train_embeddings = model.encode(list(data["train"]["text"]), show_progress_bar=True)
test_embeddings = model.encode(list(data["test"]["text"]), show_progress_bar=True)

In [ ]:
train_embeddings.shape

In [ ]:
from sklearn.linear_model import LogisticRegression

# 훈련 세트의 임베딩으로 로지스틱 회귀 모델을 훈련합니다.
clf = LogisticRegression(random_state=42)
clf.fit(train_embeddings, data["train"]["label"])

In [ ]:
# 테스트 세트 임베딩에 대해 예측을 수행합니다.
y_pred = clf.predict(test_embeddings)
evaluate_performance(data["test"]["label"], y_pred)

**팁!**  

분류기를 사용하지 않는다면 어떻게 할 수 있을까요? 분류기 대신 클래스별 임베딩을 평균하고 코사인 유사도를 적용하여 문서와 가장 잘 맞는 클래스를 예측할 수 있습니다.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics.pairwise import cosine_similarity

# 타깃 레이블에 대한 문서의 임베딩을 모두 평균하여 타깃 임베딩을 만듭니다.
df = pd.DataFrame(np.hstack([train_embeddings, np.array(data["train"]["label"]).reshape(-1, 1)]))
averaged_target_embeddings = df.groupby(768).mean().values

# 테스트 임베딩과 가장 가까운 타깃 임베딩을 찾습니다.
sim_matrix = cosine_similarity(test_embeddings, averaged_target_embeddings)
y_pred = np.argmax(sim_matrix, axis=1)

# 모델을 평가합니다.
evaluate_performance(data["test"]["label"], y_pred)

## 데이터에 레이블이 없는 경우

In [ ]:
# 레이블의 임베딩을 만듭니다.
label_embeddings = model.encode(["A negative review",  "A positive review"])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# 각 문서와 가장 잘 맞는 레이블을 찾습니다.
sim_matrix = cosine_similarity(test_embeddings, label_embeddings)
y_pred = np.argmax(sim_matrix, axis=1)

In [ ]:
evaluate_performance(data["test"]["label"], y_pred)

**팁!**  

설명을 다르게 하면 어떨까요? **"A very negative movie review"**와 **"A very positive movie review"**를 사용하면 어떻게 되는지 확인해 보세요!

In [ ]:
label_embeddings = model.encode(["A very negative movie review",
                                 "A very positive movie review"])

sim_matrix = cosine_similarity(test_embeddings, label_embeddings)
y_pred = np.argmax(sim_matrix, axis=1)

evaluate_performance(data["test"]["label"], y_pred)

# 생성 모델로 텍스트 분류하기

## T5 모델 사용하기

In [ ]:
# 모델을 로드합니다.
pipe = pipeline(
    "text2text-generation",
    model="google/flan-t5-small",
    device="cuda:0"
)

In [ ]:
# 프롬프트를 추가합니다.
prompt = "Is the following sentence positive or negative? "
data = data.map(lambda example: {"t5": prompt + example['text']})
data

In [ ]:
# 추론을 실행합니다.
y_pred = []
for output in tqdm(pipe(KeyDataset(data["test"], "t5")), total=len(data["test"])):
    text = output[0]["generated_text"]
    y_pred.append(0 if text == "negative" else 1)

In [ ]:
evaluate_performance(data["test"]["label"], y_pred)

## ChatGPT로 분류하기

In [ ]:
# httpx 패키지의 proxies 매개변수 오류를 피하기 위해
# https://community.openai.com/t/error-with-openai-1-56-0-client-init-got-an-unexpected-keyword-argument-proxies/
!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet

In [ ]:
import openai

# 클라이언트를 만듭니다.
client = openai.OpenAI(api_key="YOUR_KEY_HERE")

In [ ]:
def chatgpt_generation(prompt, document, model="gpt-4o-mini"):
    """프롬프트와 문서를 입력받아 출력을 생성합니다."""
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant."
            },
        {
            "role": "user",
            "content":   prompt.replace("[DOCUMENT]", document)
            }
    ]
    chat_completion = client.chat.completions.create(
      messages=messages,
      model=model,
      temperature=0
    )
    return chat_completion.choices[0].message.content

In [ ]:
# 프롬프트 템플릿을 정의합니다.
prompt = """Predict whether the following document is a positive or negative movie review:

[DOCUMENT]

If it is positive return 1 and if it is negative return 0. Do not give any other answers.
"""

# Predict the target using GPT
document = "unpretentious , charming , quirky , original"
chatgpt_generation(prompt, document)

다음 단계는 전체 테스트 데이터로 오픈AI 모델 중 하나를 실행하는 것입니다. 하지만 전체 테스트 데이터는 1,066개이므로 충분한 크레딧이 있을 때만 실행하세요.

In [ ]:
# 무료 크레딧을 아끼고 싶다면 이 코드를 건너 뛰세요.
predictions = [chatgpt_generation(prompt, doc) for doc in tqdm(data["test"]["text"])]

In [ ]:
# 예측을 저장합니다.
y_pred = [int(pred) for pred in predictions]

# 성능을 평가합니다.
evaluate_performance(data["test"]["label"], y_pred)